In [17]:
import pandas as pd
import requests
# import xlrd
import json
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
input_dir = 'input_data'
landing_dir = 'landing_data'
cleaned_dir = 'cleaned_data'

In [18]:
#import data from url to get the state abbr
url="http://worldpopulationreview.com/static/states/name-abbr.json"
d = requests.get(url).json()

states_abbr = pd.DataFrame(d.items() , columns=["State","State Abbr"])

states_abbr.head()

,State,State Abbr
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [19]:
#2016 Adult Asthma
url_2016 = 'https://www.cdc.gov/asthma/brfss/2016/tableL1.htm'

df_2016 = pd.read_html(url_2016)

df_asthma_2016=df_2016[0]

asthma_2016 =df_asthma_2016.iloc[1:]

#list(asthma_2016.columns)

asthma_2016 = asthma_2016.rename(columns={'Sample Size §':'Sample Size'})
asthma_2016.drop(asthma_2016.index[51], inplace=True)
asthma_2016.drop(asthma_2016.index[53], inplace=True)
asthma_2016.drop(asthma_2016.index[52], inplace=True)
asthma_2016.drop(asthma_2016.index[51], inplace=True)

asthma_2016['Year']='2016'
asthma_2016.head()

,State,Sample Size,Prevalence (percent),Standard Error,95% CI* (percent),|| || ||,Prevalence (number),95% CI* (number),Year
1,AL,7015.0,14.9,0.60,(13.8–16.1),||,563908.0,"(518,048–609,768)",2016
2,AK,2906.0,14.2,1.08,(12.1–16.4),||,78995.0,"(67,042–90,948)",2016
3,AZ,10903.0,14.6,0.57,(13.5–15.7),||,771201.0,"(709,808–832,595)",2016
4,AR,5279.0,14.3,0.91,(12.6–16.1),||,327559.0,"(285,053–370,066)",2016
5,CA,11370.0,12.8,0.39,(12.1–13.6),||,3895034.0,"(3,657,836–4,132,232)",2016


In [20]:
#2015 Adult Asthma
url_2015 = 'https://www.cdc.gov/asthma/brfss/2015/tableL1.html'

df_2015 = pd.read_html(url_2015)

df_asthma_2015=df_2015[0]

asthma_2015 =df_asthma_2015.iloc[1:]
asthma_2015.drop(asthma_2015.index[51], inplace=True)
asthma_2015.drop(asthma_2015.index[51], inplace=True)
asthma_2015.drop(asthma_2015.index[51], inplace=True)

asthma_2015['Year']='2015'
asthma_2015.head()

C:\Users\tmher\Anaconda3\envs\pandas\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,State,Sample Size,Prevalence (percent),Standard Error,95% CI* (percent),|| || ||,Prevalence (number),95% CI* (number),Year
1,AL,7924.0,15.6,0.58,(14.5 – 16.8),||,586899.0,"(542,879 – 630,918)",2015
2,AK,3642.0,13.4,1.02,(11.4 – 15.4),||,73931.0,"(62,413 – 85,448)",2015
3,AZ,7912.0,15.7,0.61,(14.5 – 16.9),||,812312.0,"(747,644 – 876,981)",2015
4,AR,5234.0,16.1,0.96,(14.2 – 18.0),||,364673.0,"(319,408 – 409,939)",2015
5,CA,12568.0,12.9,0.39,(12.1 – 13.6),||,3859786.0,"(3,626,298 – 4,093,275)",2015


In [21]:
#2014 Adult Asthma
url_2014 = 'https://www.cdc.gov/asthma/brfss/2014/tableL1.htm'

df_2014 = pd.read_html(url_2014)

df_asthma_2014=df_2014[0]

asthma_2014 =df_asthma_2014.iloc[1:]
asthma_2014.drop(asthma_2014.index[51], inplace=True)
asthma_2014.drop(asthma_2014.index[51], inplace=True)
asthma_2014.drop(asthma_2014.index[51], inplace=True)

asthma_2014['Year']='2014'
asthma_2014.head()

C:\Users\tmher\Anaconda3\envs\pandas\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,State,Sample Size,Prevalence (percent),Standard Error,95% CI* (percent),|| || ||,Prevalence (number),95% CI* (number),Year
1,AL,8620.0,14.6,0.56,(13.5 – 15.7),||,542973.0,"(500,848 – 585,098)",2014
2,AK,4375.0,12.4,0.70,(11.1 – 13.8),||,68955.0,"(61,153 – 76,756)",2014
3,AZ,14821.0,14.3,0.49,(13.3 – 15.3),||,725658.0,"(674,592 – 776,723)",2014
4,AR,5245.0,13.3,0.74,(11.9 – 14.8),||,300882.0,"(267,523 – 334,240)",2014
5,CA,8806.0,12.7,0.44,(11.8 – 13.5),||,3726172.0,"(3,466,949 – 3,985,395)",2014


In [22]:
#2013 Adult Asthma
url_2013 = 'https://www.cdc.gov/asthma/brfss/2013/tableL1.htm'

df_2013 = pd.read_html(url_2013)

df_asthma_2013=df_2013[0]

asthma_2013 =df_asthma_2013.iloc[1:]
#asthma_2013[asthma_2013.State != 'Territories']
asthma_2013.drop(asthma_2013.index[51], inplace=True)
asthma_2013.drop(asthma_2013.index[51], inplace=True)
asthma_2013.drop(asthma_2013.index[51], inplace=True)
#asthma_2013=asthma_2013.drop('Territories')

asthma_2013['Year']='2013'
asthma_2013.head()

C:\Users\tmher\Anaconda3\envs\pandas\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,State,Sample Size,Prevalence (percent),Standard Error,95% CI* (percent),|| || ||,Prevalence (number),95% CI* (number),Year
1,AL,6484.0,13.6,0.61,(12.4 – 14.8),||,504811.0,"(459,449 – 550,172)",2013
2,AK,4564.0,14.8,0.83,(13.2 – 16.4),||,81577.0,"(72,193 – 90,961)",2013
3,AZ,4236.0,14.6,0.90,(12.9 – 16.4),||,730378.0,"(642,838 – 817,919)",2013
4,AR,5248.0,13.1,0.69,(11.8 – 14.5),||,294578.0,"(263,444 – 325,712)",2013
5,CA,11500.0,14.4,0.44,(13.5 – 15.2),||,4198521.0,"(3,940,609 – 4,456,432)",2013


In [23]:
combined_df=pd.concat([asthma_2013, asthma_2014,asthma_2015, asthma_2016])
combined_df=combined_df.sort_values(by=['State', 'Year'])
combined_df=combined_df.drop(['|| || ||', 'Prevalence (number)', '95% CI* (number)'], axis=1)
combined_df=combined_df.set_index('State')
combined_df=combined_df.rename(columns={'Sample Size':'Asthma Sample Size', 'Prevalence (percent)':'Asthma Prevalence (%)', '95% CI* (percent)':'95% CI* (%)'})
combined_df.head()

,Asthma Sample Size,Asthma Prevalence (%),Standard Error,95% CI* (%),Year
State,,,,,
AK,4564.0,14.8,0.83,(13.2 – 16.4),2013
AK,4375.0,12.4,0.70,(11.1 – 13.8),2014
AK,3642.0,13.4,1.02,(11.4 – 15.4),2015
AK,2906.0,14.2,1.08,(12.1–16.4),2016
AL,6484.0,13.6,0.61,(12.4 – 14.8),2013


In [24]:
output_file = os.path.join( landing_dir, 'Asthma.csv')
combined_df.to_csv(output_file, index=False)